## First import dependencies and data from SteevEbu.preproc

In [16]:
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../notebooks/processed_data_steeve_updated.csv')

In [3]:
df = df.dropna().reset_index().drop(columns='index')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61442 entries, 0 to 61441
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Unnamed: 0  61442 non-null  int64   
 1   content     61442 non-null  object  
 2   sentiment   61442 non-null  category
 3   code        61442 non-null  int8    
dtypes: category(1), int64(1), int8(1), object(1)
memory usage: 1.1+ MB


## Dataset is much too large, so at the moment I am using a 5% sample to test the model.

In [30]:
df = df.sample(frac=0.05)

## Encode the categories numerically

In [31]:
df.sentiment = pd.Categorical(df.sentiment)
df['code'] = df.sentiment.cat.codes

## Create variables, and split into train and test data

In [33]:
y = df.code
X = df.content

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)

In [35]:
X_train

43117           carterronson going im overseas u early tho
53694    starting integrating twitter forexforumsorg lo...
48493     gfalcone hmmm toughie haha alike tom question xx
14651                                  feeling overwhelmed
8379               feel agitated become easily overwhelmed
                               ...                        
18814    feel nay gorgeous right track getting good gra...
8001     leave sunday feeling utterly drained ounce any...
54581    chrisfromracine thats sure thats intentionally...
28030       shadowfish id drive get taxi pool tablejukebox
31123     throat infection coming strong think taste blood
Name: content, Length: 2150, dtype: object

## Custom transformer courtesy of Christophe to ensure the model has the correct input matrix

In [44]:
class ToArray(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.toarray()

## Pipeline everything

In [38]:
# Create Pipeline
pipeline = make_pipeline(
    CountVectorizer(),
    ToArray(),
    GaussianNB()
)

# Set parameters to search
#parameters = {
   # 'tfidf__ngram_range': ((1,1), (2,2)),
   # 'nb__alpha': (0.1,1),}
# Perform grid search
# grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, 
                           #verbose=1, scoring = "accuracy", 
                           #refit=True, cv=5)
# grid_search.fit(X_train,y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('toarray', <__main__.ToArray object at 0x11b073a60>),
                ('gaussiannb', GaussianNB())])

In [39]:
y_pred = pipeline.predict(X_test)